### final

In [86]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 수
sql = '''
select count(mgm_bldrgst_pk), sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu
  from m1.bld_rgst
 where main_purps_cd_nm not in 
	(
	select distinct main_purps_cd_nm
	from m1.bld_rgst
	where main_purps_cd_nm like '%주택%'
	)
group by sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['bld_count','pnu']

# pnu별 주용도명
sql = '''
select main_purps_cd_nm, sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu
  from m1.bld_rgst
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_principal_use = pd.DataFrame(rows)
df_bld_principal_use.columns = ['principal_use','pnu']

# pnu별 유동인구 
sql = 'select pnu, f_tot from m2.cremao_land_floating_population'
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['pnu','f_tot']

# 데이터 프레임 병합
df = pd.merge(df_f_tot, df_bld_rgst, on='pnu', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['bld_count'].isnull()]

# 상업용 부동산 수 / 유동인구
df['commercial_per_population'] = (df['bld_count'] / df['f_tot'])

# 주용도코드 붙이기
df = pd.merge(df, df_bld_principal_use, on='pnu', how='left')
df = df.drop_duplicates('pnu').reset_index(drop=True)

# inf 값 제거후 컬럼 정리
df = df[~(df['commercial_per_population']==np.inf)]
df = df[['pnu', 'principal_use', 'commercial_per_population']]
df['base_ym'] = datetime.today().strftime("%Y%m")


### 2. DB INSERT
import datetime

table = 'm2.datadam_commercial_per_population'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_commercial_per_population;
    create table m2.datadam_commercial_per_population (    
    pnu varchar(19)
    , principal_use varchar(200)
    , commercial_per_population double precision
    , base_ym varchar(6)
    )
    distkey(pnu)
    compound sortkey(principal_use, base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-31 20:55:37.600379  :  0
2022-10-31 20:55:42.790020  :  10000
2022-10-31 20:55:45.006533  :  20000
2022-10-31 20:55:50.336306  :  30000
2022-10-31 20:55:53.035980  :  40000
2022-10-31 20:55:56.007491  :  50000
2022-10-31 20:55:58.410915  :  60000
2022-10-31 20:56:00.948826  :  70000
2022-10-31 20:56:03.354175  :  80000
2022-10-31 20:56:05.701640  :  90000
2022-10-31 20:56:08.260658  :  100000
2022-10-31 20:56:14.029132  :  110000
2022-10-31 20:56:20.077323  :  120000
2022-10-31 20:56:25.197318  :  130000
2022-10-31 20:56:27.994698  :  140000
2022-10-31 20:56:30.517375  :  150000
2022-10-31 20:56:32.924666  :  160000
2022-10-31 20:56:35.979257  :  170000
2022-10-31 20:56:38.493691  :  180000
2022-10-31 20:56:43.719379  :  190000
2022-10-31 20:56:46.405183  :  200000
2022-10-31 20:56:48.821786  :  210000
2022-10-31 20:56:51.410271  :  220000
2022-10-31 20:56:53.889993  :  230000
2022-10-31 20:56:56.675226  :  240000
2022-10-31 20:57:01.303829  :  250000
2022-10-31 20:57:03.812536

In [83]:
df.describe()

,commercial_per_population
count,462282.000000
mean,0.000857
std,0.032387
min,0.000000
25%,0.000007
50%,0.000015
75%,0.000057
max,18.000000


In [84]:
df

,pnu,principal_use,commercial_per_population,base_ym
0,1111010800100100003,제2종근린생활시설,0.000006,202210
1,1111010800100210001,제1종근린생활시설,0.000006,202210
2,1111010800100310002,근린생활시설,0.000012,202210
3,1111010800100310025,제1종근린생활시설,0.000006,202210
4,1111010800100370002,단독주택,0.000012,202210
...,...,...,...,...
462283,4380036029102260001,제2종근린생활시설,0.081081,202210
462284,4380036029103040002,창고시설,0.006652,202210
462285,4380036029103340000,관광휴게시설,0.013304,202210
462286,4380036033107370004,제2종근린생활시설,0.008772,202210


###### ver1 (법정동 단위)

In [38]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 법정동별 상가 면적
sql = '''
select sigungu_cd||bjdong_cd as pnu, count(mgm_bldrgst_pk) as total_count
  from m1.bld_rgst
 where main_purps_cd_nm not in 
    (
    select distinct main_purps_cd_nm
    from m1.bld_rgst
    where main_purps_cd_nm like '%주택%'
    )
group by sigungu_cd||bjdong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['emd_cd','total_count']

# 법정동별 유동인구 
sql = '''
    select substring (pnu, 0, 11), sum(f_tot)
      from m2.cremao_land_floating_population
     group by substring (pnu, 0, 11)
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['emd_cd','f_tot']
df_f_tot = df_f_tot[~df_f_tot['f_tot'].isnull()].reset_index(drop=True)

# 데이터 프레임 병합
df = pd.merge(df_f_tot, df_bld_rgst, on='emd_cd', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['total_count'].isnull()]

# 상가면적 / 거주인구
df['commercial_per_population'] = (df['total_count'] / df['f_tot'])
df = df[['emd_cd','commercial_per_population']]
df['base_ym'] = datetime.today().strftime("%Y%m")


### 2. DB INSERT
import datetime

table = 'm2.datadam_commercial_per_population'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_commercial_per_population;
    create table m2.datadam_commercial_per_population (    
    emd_cd varchar(10)
    , commercial_per_population double precision
    , base_ym varchar(6)
    )
    distkey(emd_cd)
    compound sortkey(base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-30 18:08:47.688140  :  0
2022-10-30 18:08:50.007249  :  10000
17296


In [37]:
df.describe()

,commercial_per_population
count,17296.000000
mean,0.051612
std,0.292162
min,0.000000
25%,0.000259
50%,0.003113
75%,0.020263
max,17.100000
